In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('train_twitter.csv')

In [3]:
test_data=pd.read_csv('test_twitter.csv')

In [4]:
y_train=data['airline_sentiment']
x_train=data['text']

In [5]:
#x_train.head(),y_train.head()

In [6]:
x_test=test_data['text']

In [7]:
#data['airline'].unique()

In [8]:
airline=['@SouthwestAir', '@united', '@JetBlue', '@USAirways', '@AmericanAir',
       '@VirginAmerica','']
tag=set(airline)

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.corpus import wordnet
import nltk

In [11]:
stops=set(stopwords.words('english'))
punct=list(string.punctuation)
stops.update(punct)
stops.update(tag)
#len(stops)

In [12]:
all_tweets=[''.join(tweet) for tweet in x_train]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv=CountVectorizer(max_features=1200,ngram_range=(1,3),stop_words=stops)

In [15]:
x_train_features=cv.fit_transform(x_train)

In [20]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
#cv.get_feature_names()

In [21]:
x_test_features=cv.transform(x_test)

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rfc=RandomForestClassifier()

In [24]:
rfc.fit(x_train_features,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
from sklearn.tree import DecisionTreeClassifier


In [26]:
DTC=DecisionTreeClassifier()
DTC.fit(x_train_features,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
from sklearn.neighbors import KNeighborsClassifier

In [28]:
knc=KNeighborsClassifier()
knc.fit(x_train_features,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [29]:
from sklearn.naive_bayes import GaussianNB

In [30]:
gnb=GaussianNB()
gnb.fit(x_train_features.toarray(),y_train)

GaussianNB(priors=None)

In [31]:
y_pred=rfc.predict(x_test_features)

In [32]:
y_pred=DTC.predict(x_test_features)

In [33]:
y_pred=knc.predict(x_test_features)

In [34]:
y_pred=gnb.predict(x_test_features.toarray())

In [35]:
np.savetxt('ff.csv',y_pred,fmt='%s')

In [36]:
documents=[ (tweet.split(" "),sentiment) for tweet,sentiment in zip(x_train,y_train)]
test_documents=[tweet.split(" ") for tweet in x_test]

In [34]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [35]:
lemmatizer=WordNetLemmatizer()

In [36]:
def clean_review(words):
    output_words=[]
    for word in words:
        if word.lower() not in stops:
            
            pos=pos_tag([word])
            clean_word=lemmatizer.lemmatize(word,get_simple_pos(pos[0][1]))
            output_words.append(clean_word)
    return output_words

In [37]:
documents=[(clean_review(document),category) for document,category in documents]
test_documents=[clean_review(document) for document in test_documents]
len(test_documents)

3660

In [38]:
all_words=[]
for document in documents:
    all_words+=document[0]

In [39]:
dictionary= nltk.FreqDist(all_words)

In [40]:
common=dictionary.most_common(2000)
features=[i[0] for i in common]

In [41]:
def get_feature_dict(words):
    current_features={}
    word_set=set(words)
    for w in features:
        current_features[w]=w in word_set
    return current_features

In [42]:
training_data=[(get_feature_dict(doc),category) for doc,category in documents]

In [43]:
testing_data=[get_feature_dict(doc) for doc in test_documents]

In [44]:
from nltk import NaiveBayesClassifier

In [45]:
clf=NaiveBayesClassifier.train(training_data)

In [46]:
clf.show_most_informative_features(15)

Most Informative Features
                    you! = True           positi : negati =     41.8 : 1.0
                     :-) = True           positi : negati =     41.8 : 1.0
               wonderful = True           positi : negati =     31.7 : 1.0
                      hr = True           negati : positi =     29.2 : 1.0
                awesome! = True           positi : negati =     29.1 : 1.0
                   THANK = True           positi : negati =     29.1 : 1.0
                favorite = True           positi : negati =     29.1 : 1.0
                    Wall = True           neutra : negati =     26.5 : 1.0
                   thank = True           positi : negati =     25.1 : 1.0
              @JetBlue's = True           neutra : negati =     24.5 : 1.0
                   &lt;3 = True           positi : negati =     24.1 : 1.0
                 Thanks! = True           positi : negati =     23.8 : 1.0
                      :) = True           positi : negati =     22.6 : 1.0

In [47]:
y_pred=[]
for i in range(len(testing_data)):
    y_pred.append(clf.classify(testing_data[i]))


In [48]:
np.savetxt('ff.csv',y_pred,fmt='%s')